In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [2]:
#!python
from numpy import *
from numpy.random import *
%matplotlib inline
import os
import time
import numpy as np
import pandas as pd
import sys
import cPickle as pickle
from   random import *
import scipy.stats
import matplotlib.pyplot as plt
sys.path.insert(1,'./code')
from visualize import plot_raw_data, get_population_data, plot_results
from EM import TransformToProb, RunEM
import pyparticleest.models.nlg as nlg
import pyparticleest.simulator as simulator
import pyparticleest.utils.kalman as kalman
from scipy.stats import bernoulli
import pyparticleest.interfaces as interfaces
from scipy.optimize import newton

//anaconda/lib/python2.7/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [3]:
df = pd.read_csv('thaldata.csv',header=None)
Y = df.values.reshape(3000)
T = len(Y)

In [4]:
# Forward particles
N = 100
# Backward trajectories
M = 30
# 
P0 = 0.1
Q = 0.1
#trial
trial = 50
#
coefficient = scipy.special.binom(trial,Y)
#
l2pi = math.log(2 * math.pi)

In [5]:
def resample(weights):
  n = len(weights)
  indices = []
  C = [0.] + [sum(weights[:i+1]) for i in range(n)]
  u0, j = random(), 0
  for u in [(u0+i)/n for i in range(n)]:
    while u > C[j]:
      j+=1
    indices.append(j-1)
  return indices

In [6]:
def measure(X, Y, trial, t, coefficient):
    # log-pdf of p(y_t|x_t)
    return np.log(coefficient[t]) + X[t,:] * Y[t] - trial * np.log(1+np.exp(X[t,:]))

In [7]:
def forward(X, a, S, t):
    #log-pdf of p(x_t|x_{t-1})
    return -0.5 * (l2pi + np.log(S) + ((X[t,:]-a).ravel() ** 2) / S)

In [8]:
def lognormpdf_scalar(err, S):
    """
    Calculate gaussian probability density of all elements in err, when
    err[i] ~ N(0,S) and each element in err is a scalar
    """
    return -0.5 * (l2pi + np.log(S) + (err.ravel() ** 2) / S)

In [9]:
def filtering(T, N, Q, trial, coefficient):
    ##Initialization
    t = 0
    X = np.empty([T,N])
    ancestor = np.empty([T,N])
    w = np.empty([T,N])
    W = np.empty([T,N])
    X[t,:] = np.random.normal(0, np.sqrt(Q), N)
    w[t,:] = measure(X, Y, trial, t, coefficient)
    W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
    W[t,:] /= np.sum(W[t,:])
    ancestor[t,:]= X[t,resample(W[t,:])]
    for tt in range(T-1):
        t=tt+1
        a = ancestor[tt,:]
        X[t,:] = a + np.random.normal(0, np.sqrt(Q), N) ##f(x_t|x_{t-1}) standard deviation
        w[t,:] = measure(X, Y, trial, t, coefficient) ##g(y_t|x_t)
        W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
        W[t,:] /= np.sum(W[t,:])
        ancestor[t,:]= X[t,resample(W[t,:])]
    
    return X, w, W, ancestor

In [13]:
def smoothing(T, N, Q, trial, coefficient, M, X, W, ancestor):
    w_back = np.empty([T,N])
    W_BACK = np.empty([T,N])
    smoother = np.empty([T,M])
    for j in range(M):
        myList = np.random.multinomial(1, W[T-1,:])
        smoother[T-1,j] = ancestor[T-1, next((i for i, x in enumerate(myList) if x), None)]
    for tt in range(T-1):
        t = T-2 - tt
        for j in range(M):
            w_back = W[t,:] * np.exp(lognormpdf_scalar(smoother[t+1,j]-ancestor[t,:], Q))
            W_BACK[t,:] = w_back/np.sum(w_back)
            myList = np.random.multinomial(1, W_BACK[t,:])
            smoother[t,j] = ancestor[t, next((i for i, x in enumerate(myList) if x), None)]
            
    return smoother

In [ ]:
#iterations
iteration=50
#initial_sigma
sigma_ls = np.empty(iteration+1)
sigma_ls[0]=0.1
for i in range(iteration):
    P0 = sigma_ls[i] * np.eye(1)    
    Q = sigma_ls[i] * np.eye(1)
    [X, w, W, ancestor] = filtering(T, N, Q, trial, coefficient)
    est_smooth = smoothing(T, N, Q, trial, coefficient, M, X, W, ancestor)
    posteriorcovariance=np.array(np.diff(est_smooth,axis=0))**2
    sigma_ls[i+1] = np.sum(posteriorcovariance/(M*T)) * np.eye(1)

In [15]:
est_smooth.shape

(3000, 30)

In [132]:
##Initialization
t = 0
v = np.zeros(N)
X = np.empty([T,N])
ancestor = np.empty([T,N])
w = np.empty([T,N])
W = np.empty([T,N])
X[t,:] = np.random.normal(0, np.sqrt(P0), N)
w[t,:] = measure(X, Y, trial, t, coefficient)
W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
W[t,:] /= np.sum(W[t,:])
ancestor[t,:]= X[t,resample(W[t,:])]

In [48]:
noise = np.random.normal(0, np.sqrt(P0), ([T,N]))

In [11]:
mode = np.empty(T)
variance = np.empty(T)

In [133]:
%%time
for tt in range(T-1):
    t=tt+1
    #samplemean = nanmean(ancestor[tt,:])
    #samplevariance = nanvar(ancestor[tt,:]) + Q
    #mode = samplemean + (samplevariance)*(Y[t] - np.exp(samplemean)/(1 + np.exp(samplemean)) * trial)
    #variance = 1.0 / (1.0/((samplevariance) + trial * np.exp(samplemean)/(1+np.exp(samplemean))**2))
    #X[t,:] = np.random.normal(mode[tt],np.sqrt(variance[tt]), N)
    #X[t,:] = mode + variance * noise[t,:]
    a = ancestor[tt,:]
    X[t,:] = a + np.random.normal(0, np.sqrt(Q), N) ##f(x_t|x_{t-1}) standard deviation
    #w = measure(X, Y, trial, t, coefficient) + forward(X, ancestor[tt,:], Q, t) - lognormpdf_scalar(X[t,:]-mode, variance)   
    w[t,:] = measure(X, Y, trial, t, coefficient) ##g(y_t|x_t)
    W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
    W[t,:] /= np.sum(W[t,:])
    ancestor[t,:]= X[t,resample(W[t,:])]

CPU times: user 2.79 s, sys: 29.6 ms, total: 2.82 s
Wall time: 2.87 s


In [56]:
maxlogweight = np.max(w,axis=1)
weight = np.exp(w - np.reshape(np.repeat(maxlogweight, N),(T, N)))
np.sum(np.log(np.sum(weight, axis = 1)/N))+np.sum(maxlogweight)

-3141.4401146831378

In [145]:
%%time
w_back = np.empty([T,N])
W_BACK = np.empty([T,N])
smoother = np.empty([T,M])
for j in range(M):
    myList = np.random.multinomial(1, W[t,:])
    smoother[T-1,j] = ancestor[T-1, next((i for i, x in enumerate(myList) if x), None)]
for tt in range(T-1):
    t = T-2 - tt
    for j in range(M):
        w_back = W[t,:] * np.exp(lognormpdf_scalar(smoother[t+1,j]-ancestor[t,:], Q))
        W_BACK[t,:] = w_back/np.sum(w_back)
        myList = np.random.multinomial(1, W_BACK[t,:])
        smoother[t,j] = ancestor[t, next((i for i, x in enumerate(myList) if x), None)]

CPU times: user 5.57 s, sys: 42 ms, total: 5.62 s
Wall time: 5.66 s


In [77]:
w_back = W[t,0] * np.exp(lognormpdf_scalar(smoother[t,0]-ancestor[t-1,:], Q))
W_BACK[t,:] = w_back/np.sum(w_back)
smoother[t-1,0] = ancestor[t-1, backward_resample(W_BACK[t,:], N, 1)]

In [159]:
df = pd.read_csv('thaldata.csv',header=None)
Y = df.values.reshape(3000)
#mat = np.loadtxt('train_data.txt')
#Y = mat[19,1:]
T = len(Y)

In [142]:
#iterations
iteration=50
#initial_sigma
sigma_ls = np.empty(iteration+1)
sigma_ls[0]=0.1

In [ ]:
#iterations
iteration=50
#initial_sigma
sigma_ls = np.empty(iteration+1)
sigma_ls[0]=0.1
for i in range(iteration):
    P0 = sigma_ls[i] * np.eye(1)    
    Q = sigma_ls[i] * np.eye(1)
    model = StdNonLin(P0, Q)
    sim = simulator.Simulator(model,u=None,y=Y)
    sim.simulate(N, M, filter='PF', smoother='full', meas_first=False)
    est_smooth = sim.get_smoothed_estimates()
    posteriorcovariance=np.array(np.diff(est_smooth[:,:,0],axis=0))**2
    sigma_ls[i+1] = np.sum(posteriorcovariance/(M*T)) * np.eye(1)

In [ ]:
print measure(X, Y, trial, 1000, coefficient)
print coefficient[1000]
print forward(X, ancestor[999,:], Q, 1000)
print lognormpdf_scalar(X[1000,:]-mode, variance)

In [ ]:
## for t > 0
for tt in range(1):
    t=tt+1
    a = ancestor[t,:]
    mode = a + t*Q*(Y[t] - np.exp(a)/(1 + np.exp(a)) * (Y[t] - trial * (1+np.exp(a))))
    variance = 1.0 / (t*Q + Y[t]*np.exp(a)/(1+np.exp(a))**2 + trial * np.exp(a)*(1-np.exp(a))/(1+np.exp(a))**3)
    X[t,:] = np.random.normal(mode,variance)
    w[t,:] = measure(X, Y, trial, t, coefficient) + forward(X, a, Q, t) - lognormpdf_scalar(X[t,:]-mode, variance)
    W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
    W[t,:] /= np.sum(W[t,:])
    ancestor[t+1,:]= X[t,resample(W[t,:])]

In [ ]:
1.0 / (t*Q + Y[t]*np.exp(a)/(1+np.exp(a))**2 + trial * np.exp(a)*(1-np.exp(a))/(1+np.exp(a))**3)

In [ ]:
np.exp(a)*(1-np.exp(a))/(1+np.exp(a))**3

In [ ]:
ancestor[1,:]

In [ ]:
signal.resample()

In [ ]:
def h(x):
    return x - ancestor + t*Q *(y[t] - np.exp(x)/(1+np.exp(x)))/(1+np.exp(x))

def dh(x):
    return 1 - t * Q

In [ ]:
def k(x):
    return x**3 -1

In [ ]:
def dk(x):
    return 3*x**2

In [ ]:
x0=0.4
print newton(k, x0, dk)

In [ ]:
X = np.empty([T,N])
ancestor = np.empty([T,N])
w = np.empty([T,N])
W = np.empty([T,N])
X[t,:] = np.random.normal(0, np.sqrt(P0), N)
w[t,:] = measure(X, Y, trial, t, coefficient)
W[t,:] = np.exp(w[t,:] - np.max(w[t,:]))
W[t,:] /= np.sum(W[t,:])
ancestor[t,:]= X[t,resample(W[t,:])]